# Dino-Nuggetology
* Team Decaffeinated – So Hirota, Penny King, Garvey Li

In this notebook you will be able to use and demo our dino nugget classification model using various photos of dino nuggets.

First let's import the necessary packages:

In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from PIL import Image
import numpy as np
from img_methods import *

import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


### Define the AutoEncoder Class

In [58]:
class Conv_AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__() 
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 4, stride = 3, padding = 1), # 32, 16, 67, 67
            nn.ReLU(),
            nn.Conv2d(8, 16, 3, stride = 2, padding = 1),# 32, 32, 34, 34
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride = 2, padding = 1), # 32, 32, 17, 17
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * 17 * 17, 3000),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3000, 32 * 17 * 17),
            nn.ReLU(),
            nn.Unflatten(dim = 1, unflattened_size=(32, 17, 17)),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride = 2, padding = 1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, 3, stride = 2, padding = 1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 1, 4, stride = 3, padding = 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

### Run Necessary Functions

In [59]:
black_border = np.zeros((200, 200))
black_border[0:10, :] = 1
black_border[190:200, :] = 1
black_border[:, 0:10] = 1
black_border[:, 190:200] = 1

def process_image(img_path):
    """
    Takes in the path of a JPG and creates a new image containing its edges.
    
    Parameters:
    img_path (str): Image path.
    
    Returns:
    img_edge (List): Edge image as a numpy array.
    
    """
    
    # Transforms to do: Mirror, rotate (15 degree increments)
    
    rotation_degrees = np.arange(0, 360, 15)
    
    gray_img = load_gray_image(img_path)
    gauss_img = gaussian_lpf(gray_img, 42 ** 2)
    
    gmag, _, _ = edge_detection(gauss_img)
    binary_gmag = (gmag > 0.65) - black_border
    original_edge = np.clip(binary_gmag * 255, 0, 255)
    #mirrored = np.flip(original_edge)
    """
    all_imgs = []
    for deg in rotation_degrees:
        all_imgs.append(imutils.rotate(original_edge, angle=deg))
    for deg in rotation_degrees:
        all_imgs.append(imutils.rotate(mirrored, angle=deg))
    """
    #return all_imgs
    
    return original_edge

class CustomDataset(Dataset):
    def __init__(self, root_dir, file_list, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.file_list[0]
        img_name = os.path.join(self.root_dir, img_name)
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image

# Define transformations to apply to your images
transform = transforms.Compose([
    # transforms.Resize((200, 200)), # not necessary, since images are already 200x200
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

def anomaly_detection(fp):
    fp = [fp]
    data = CustomDataset(file_list = fp, root_dir = 'C:\\Users\\So\\Documents\\code\\datahacks_2024\\demo_imgs', transform=transform)

    dataloader = DataLoader(data, batch_size=1, shuffle=False)
    ae = Conv_AutoEncoder()
    ae.load_state_dict(torch.load('C:\\Users\\So\\Documents\\code\\datahacks_2024\\classification\\validated_conv_autoencoder_50_epochs.pth'))
    
    res = {}
    for img in dataloader:
        # Pass the data through the model
        with torch.no_grad():  # No need to track gradients during inference
            recon = ae(img)
        # recon = ae(img)
        res['og'] = img
        res['recon'] = recon
    
    og = 0
    recon = 1

    original = res['og'][0][0]
    recon = res['recon'][0][0]
    mse = F.mse_loss(original, recon)

    threshold = (0.004964645775481617 + 0.007269915045952306) / 2
    res = ''
    if threshold > mse:
        res = 'NORMAL'
    else:
        res = 'MALFORMED'
    print(f'{res}: reconstruction error: {mse:4f}')

Now we can use the model.

### Tutorial
Here is an picture of a triceratops dino nugget at the path `demo_imgs/pte_demo.jpg`

<img src='../demo_imgs/pte_demo.jpg' height = 200 width = 200/>

Let's use the model to classify the dino species.

In [60]:
img = process_image('C:\\Users\\So\\Documents\\code\\datahacks_2024\\demo_imgs\\pte_demo.jpg')
Image.fromarray(img).convert('RGB').save('C:\\Users\\So\\Documents\\code\\datahacks_2024\\demo_imgs\\temp.jpeg')

anomaly_detection('temp.jpeg')

NORMAL: reconstruction error: 0.004169


Here is another example using a deformed dino nugget.

Below is the picture we will use. The path is: `demo_imgs/deformed_nugget.jpg`

<img src='../demo_imgs/deformed_nugget.jpg' height = 200 width = 200/>


In [61]:
img = process_image('C:\\Users\\So\\Documents\\code\\datahacks_2024\\demo_imgs\\deformed_nugget.jpg')
Image.fromarray(img).convert('RGB').save('C:\\Users\\So\\Documents\\code\\datahacks_2024\\demo_imgs\\temp.jpeg')

anomaly_detection('temp.jpeg')

MALFORMED: reconstruction error: 0.006177


### Your Turn!
Feel free to try on your own now. You can find various different images to use for the model in the folder `demo_imgs\`

In [ ]:
# your code here